# Introduction

We intend to produce a model based on BERT to infer capacity of predicting if a tweet can be categorized as an Alert disaster tweet or not.

Some kernels have already implemented such models :

* egortrushin/nlp-with-disaster-tweets-roberta-using-pytorch         NLP with Disaster Tweets: RoBERTa using PyTorch     Egor Trushin

* ashishsingh226/disaster-tweet-classification-using-distil-bert     Disaster Tweet Classification using Distil Bert     AshishSingh226


In a perspective to contribute to the community, we propose another implementation for BERT with fast-bert api on PyTorch.

The objective is to try achieve a high accuracy scoring by :

* implementing most appropriate pretrained model
* fine tuning hyperparameters
* fine tune optimizer  

# Imports

In [ ]:
!pip install fast-bert

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

import torch
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner

from fast_bert.metrics import fbeta, roc_auc, accuracy
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging
import os
import pickle
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
import re
import emoji

import warnings
warnings.filterwarnings('ignore')

logger = logging.getLogger()
device_cuda = torch.device("cuda")

# Datasets loading

In [ ]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv',sep=",")

In [ ]:
test_data = pd.read_csv('../input/nlp-getting-started/test.csv',sep=",")

In [ ]:
print("Total rows in train data: ",train_data.shape[0])
print("Total columns in train data: ",train_data.shape[1])
print("-"*30)
print("Total rows in test data: ",test_data.shape[0])
print("Total columns in test data: ",test_data.shape[1])

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
train_data.head()

# Data Exploration & Cleaning

In [ ]:
print(train_data.isnull().sum())
print("-"*30)
print(test_data.isnull().sum())

We drop keyword and location columns as too many Nan and not really useful in the perspective of this classification objective.

In [ ]:
train_data.groupby(train_data.target).count().text

In [ ]:
sns.countplot(train_data.target,data = train_data)
plt.show()

The dataset is quite balanced between the 2 classes which is ok for our classification purpose.

### Dropping keyword and location columns

In [ ]:
train_data = train_data.drop(['keyword','location'], axis = 1)
test_data = test_data.drop(['keyword','location'], axis = 1)

In [ ]:
train_data.head()

# Preprocessing 

We need to remove superfluous data like url, emoji

In [ ]:
def cleanTweet(txt):
    txt = re.sub(r'@[A-Za-z0-9_]+','',txt)
    txt = re.sub(r'#','',txt)
    txt = re.sub(r'RT : ','',txt)
    txt = re.sub(r'\n','',txt)
    # to remove emojis
    txt = re.sub(emoji.get_emoji_regexp(), r"", txt)
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+','',txt)
    txt = re.sub(r"https?://\S+|www\.\S+","",txt)
    txt = re.sub(r"<.*?>","",txt)
    return txt  

def make_Lower(text):
    return str.lower(text)

In [ ]:
train_data.text = train_data.text.apply(cleanTweet)
train_data.text = train_data.text.apply(make_Lower)

test_data.text = test_data.text.apply(cleanTweet)
test_data.text = test_data.text.apply(make_Lower)

We save these preprocessed datasets as they will be model input from disk resources

In [ ]:
train_data.to_csv('train_preprocessed.csv', index=False) 
test_data.to_csv('test_preprocessed.csv', index=False) 

# Model

### Define main paths

In [ ]:
DATA_PATH = Path('data/')
LABEL_PATH = Path('data/')
MODEL_PATH=Path('models/')
LOG_PATH=Path('logs/')
MODEL_PATH.mkdir(exist_ok=True)
DATA_PATH.mkdir(exist_ok=True)
LOG_PATH.mkdir(exist_ok=True)

### Prepare train and validation sets

In [ ]:
val_set = train_data.sample(frac=0.2, replace=False, random_state=42)
train_set = train_data.drop(index = val_set.index)
print('Number of tweets in val_set:',len(val_set))
print('Number of tweets in train_set:', len(train_set))
val_set.to_csv(str(DATA_PATH) + '/val_set.csv')
train_set.to_csv(str(DATA_PATH) + '/train_set.csv')

In [ ]:
labels = train_data.columns[2:].to_list()
with open(str(LABEL_PATH) + '/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [ ]:
all_texts = train_data['text'].to_list()
print('Number of tweets:', len(all_texts))

### Create LMDataBunch

In this phase we first fine tune BERT model to our corpus.

We first need to instanciate data bunches for batching learning phase.

Note here we use 'bert-base-uncased' tokenizer defined on pretrained BERT model. 

In [ ]:
t0 = time()
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=Path(DATA_PATH),
                    text_list=all_texts,
                    tokenizer='bert-base-uncased',
                    batch_size_per_gpu=16,
                    max_seq_length=512,
                    multi_gpu=False,
                    model_type='bert',
                    logger=logger)
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')


### Create LMLearner

We instanciate a learner for fine tuning the model based on pretrained bert model.

Here we choose simple 'bert' model, and accuracy as metric of learning process.

In [ ]:
t0 = time()
metrics = []
metrics.append({'name': 'accuracy', 'function': accuracy})
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='bert-base-uncased',
                            output_dir=Path(MODEL_PATH),
                            metrics=metrics,
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            fp16_opt_level="O2")
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

### Fitting the learner

We choose AdamW with warmup_cosine that variates learning rate from its original value to 0 with cosine periodicity.

In [ ]:

lm_learner.fit(epochs=5,
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [ ]:
from time import time
t0 = time()
lm_learner.validate()
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

In [ ]:
t0 = time()
lm_learner.save_model()
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

### Create databunch for our classification

In [ ]:
labels_col = ['target']

In [ ]:
t0 = time()
databunch = BertDataBunch(Path(DATA_PATH), Path(LABEL_PATH),
                          tokenizer='bert-base-uncased',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='text',
                          label_col=labels_col,
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='bert')
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

### Création de Learner pour la classification

Prepare metrics output with accuracy

In [ ]:
metrics = []
metrics.append({'name': 'accuracy', 'function': accuracy})


Create tensorboard out dir as it is not created straightly

In [ ]:
OUTPUT_DIR = Path(str(MODEL_PATH) + '/finetuned_model')
WGTS_PATH = Path(str(MODEL_PATH) + '/model_out/pytorch_model.bin')
FINETUNED_MODEL_PATH=Path('models/finetuned_model')
FINETUNED_MODEL_PATH.mkdir(exist_ok=True)
FINETUNED_MODEL_TENSORBOARD_PATH = Path(str(MODEL_PATH) + '/finetuned_model/tensorboard')
OUTPUT_DIR.mkdir(exist_ok=True)
FINETUNED_MODEL_TENSORBOARD_PATH.mkdir(exist_ok=True)

In [ ]:
t0 = time()
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path=str(MODEL_PATH) + '/model_out',
                        metrics=metrics,
                        device=device_cuda,
                        logger=logger,
                        output_dir=Path(OUTPUT_DIR),
                        finetuned_wgts_path=Path(WGTS_PATH),
                        warmup_steps=50,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=True,
                        logging_steps=50)
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

### Find the optimal learning rate

Based on integrated Bert Adam optimizer we try to fit the most suitable learning rate for our fine-tuning. 

This step is important as it is fine tuning, we don't want to retrain the whole model but be as accurate as possible.

In [ ]:
#cl_learner.lr_find(start_lr=1e-5, optimizer_type='adamw',cache_dir=None)

In [ ]:
#cl_learner.plot()

In [ ]:
t0 = time()
cl_learner.fit(epochs=5,
            lr=9e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

In [ ]:
t0 = time()
cl_learner.validate()
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

In [ ]:
t0 = time()
cl_learner.save_model()
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')

In [ ]:
metrics

### Prédictions

In [ ]:
predictor = BertClassificationPredictor(
                model_path=str(MODEL_PATH) + '/finetuned_model/model_out',
                label_path=str(LABEL_PATH),
                multi_label=True,
                model_type='bert',
                do_lower_case=False)

### unit test

In [ ]:
t0 = time()
tweet = test_data['text'][0]
t = predictor.predict(tweet)
t1 = time()
total_time = t1 - t0
print('total time for operation: ' + str(total_time) + 's')
for pred in t:
    print(pred[0])

### general prediction

In [ ]:
predictions = []
for idx, row in test_data.iterrows():
    t = predictor.predict(row.text)
    for pred in t:
        print(pred[0])
        predictions.append([row.id, pred[0]])

df = pd.DataFrame(predictions, columns=['id','target'])        
df.to_csv('Submission.csv', index=False)        